<a href="https://colab.research.google.com/github/vikaskapur/unsupervised-passage-reranking/blob/my-test/Zeroshot_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/vikaskapur/unsupervised-passage-reranking

Cloning into 'unsupervised-passage-reranking'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 43 (delta 13), reused 38 (delta 8), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [7]:
%cd unsupervised-passage-reranking

/content/unsupervised-passage-reranking


In [12]:
!python utils/download_data.py --resource "data.wikipedia-split.psgs_w100"

Requested resource from %s https://www.dropbox.com/s/bezryc9win2bha1/psgs_w100.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/unsupervised-passage-reranking/downloads/data/wikipedia-split/psgs_w100.tsv

Downloaded to %s /content/unsupervised-passage-reranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
Uncompressing %s /content/unsupervised-passage-reranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
 Saved to %s ./downloads/data/wikipedia-split


In [13]:
!python utils/download_data.py --resource "data.retriever-outputs.bm25.nq-test" 

Requested resource from %s https://www.dropbox.com/s/ml2lnt34ktjgft6/nq-test.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/unsupervised-passage-reranking/downloads/data/retriever-outputs/bm25/nq-test.json

Downloaded to %s /content/unsupervised-passage-reranking/downloads/data/retriever-outputs/bm25/nq-test.tar.gz
Uncompressing %s /content/unsupervised-passage-reranking/downloads/data/retriever-outputs/bm25/nq-test.tar.gz
 Saved to %s ./downloads/data/retriever-outputs/bm25


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
hf_model_name = "bigscience/T0_3B"

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(hf_model_name)
model = T5ForConditionalGeneration.from_pretrained(hf_model_name)